In [15]:
# XGBoost on Otto dataset, Tune n_estimators
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier 



from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot

import os
import time
from datetime  import datetime

In [16]:
strategyName='Long'

pahtname=r'D:\DataSets\ML_LS\ML-Long_S50M15_07To2Q20-Train_Norm.csv'

filepath= os.path.abspath(pahtname)

dataset =pd.read_csv(filepath,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
dataset.drop(columns=['open','high','low','close'],inplace=True)

dataset.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70896 entries, 2007-01-03 09:45:00 to 2020-06-30 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             70896 non-null  float64
 1   indy_ma-1100            70896 non-null  float64
 2   indy_hh-550             70896 non-null  float64
 3   indy_ll-550             70896 non-null  float64
 4   indy_mid-550            70896 non-null  float64
 5   indy_hh2-1100           70896 non-null  float64
 6   indy_ll2-1100           70896 non-null  float64
 7   indy_mid2-1100          70896 non-null  float64
 8   indy_macd110-440        70896 non-null  float64
 9   indy_signal110-440-110  70896 non-null  float64
 10  indy_hist_macd110-440   70896 non-null  float64
 11  indy_rsi25-ma20         70896 non-null  float64
 12  indy_6ATRTrail_DC-110   70896 non-null  float64
 13  cate_3trend-550_ma110   70896 non-null  int64  
 14  cat

In [17]:
def CreateData(dfx):
 labelCol=dfx.columns[len(dfx.columns)-1]
 
 X_df_temp = dfx.drop(columns=labelCol)
 Y_df_temp = dfx[labelCol]

 Xtemp=X_df_temp.to_numpy()
 ytemp=Y_df_temp.to_numpy()

 return X_df_temp,Y_df_temp, Xtemp,ytemp

In [18]:
#My Train
st='01-2007'
ed='12-2020'
df_train=dataset[st:ed]
#df_train.info()
X_df,Y_df,X_train,y_train=CreateData( df_train)

print(X_df.info())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70896 entries, 2007-01-03 09:45:00 to 2020-06-30 16:45:00
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             70896 non-null  float64
 1   indy_ma-1100            70896 non-null  float64
 2   indy_hh-550             70896 non-null  float64
 3   indy_ll-550             70896 non-null  float64
 4   indy_mid-550            70896 non-null  float64
 5   indy_hh2-1100           70896 non-null  float64
 6   indy_ll2-1100           70896 non-null  float64
 7   indy_mid2-1100          70896 non-null  float64
 8   indy_macd110-440        70896 non-null  float64
 9   indy_signal110-440-110  70896 non-null  float64
 10  indy_hist_macd110-440   70896 non-null  float64
 11  indy_rsi25-ma20         70896 non-null  float64
 12  indy_6ATRTrail_DC-110   70896 non-null  float64
 13  cate_3trend-550_ma110   70896 non-null  int64  
 14  cat

In [19]:
X_df.tail(5)

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2020-06-30 15:45:00,1.03,1.0,1.11,0.97,0.0,1.11,0.90,0.0,-33.34,-22.94,-10.41,57.53,1.00,2,1,4,3
2020-06-30 16:00:00,1.03,1.0,1.11,0.97,0.0,1.11,0.90,0.0,-33.44,-23.14,-10.30,57.88,1.00,2,1,4,3
2020-06-30 16:15:00,1.03,1.0,1.11,0.97,0.0,1.11,0.90,0.0,-33.54,-23.34,-10.20,57.93,1.00,2,1,4,3
2020-06-30 16:30:00,1.04,1.0,1.11,0.98,0.0,1.11,0.91,0.0,-33.66,-23.54,-10.12,57.28,1.01,2,1,4,3
2020-06-30 16:45:00,1.04,1.0,1.12,0.98,0.0,1.12,0.91,0.0,-33.78,-23.74,-10.04,56.61,1.01,2,1,4,3


In [20]:
Y_df.value_counts()

0    41328
1    29568
Name: LongSignal, dtype: int64

# Single Tune

In [21]:
#tuneName='n_estimators'
tuneName='max_depth'
#tuneName='learning_rate'

#scoreX="neg_log_loss"
scoreX='accuracy'


# XGBClassifier change paramter consistently

In [22]:
xseed=1992
splitN=5
option=1
if option==1 :
 kfold =  TimeSeriesSplit(n_splits=splitN,max_train_size=None)
else:
 kfold = StratifiedKFold(n_splits=splitN, shuffle=false, random_state=xseed)

In [23]:
xDept=8
xLr=0.05
xEstimators=160

print(tuneName)
if tuneName=='n_estimators':
 #paramToTuneList = range(40, 320, 10)
 paramToTuneList = [40, 80, 120, 160]
 param_grid = dict(n_estimators=paramToTuneList)
 model = XGBClassifier(max_depth=xDept,learning_rate=xLr,importance_type='gain')

elif tuneName=='max_depth':
 paramToTuneList = [3, 6, 8, 10,12,16,20]
 param_grid = dict(max_depth=paramToTuneList)
 model = XGBClassifier(n_estimators=xEstimators,learning_rate=xLr,importance_type='gain')

elif tuneName=='learning_rate':
 paramToTuneList = [0.001,0.005,0.01,0.05,0.1,0.5,1]
 param_grid = dict(learning_rate=paramToTuneList)
 model = XGBClassifier(n_estimators=xEstimators,max_depth=xDept,importance_type='gain')


grid_search = GridSearchCV(model, param_grid, scoring=scoreX, n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
grid_result

max_depth


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.05, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=160, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
       

In [24]:
grid_result.cv_results_

{'mean_fit_time': array([ 8.97445297, 19.36330843, 25.92017274, 31.46965404, 34.26976485,
        34.31358342, 28.30049062]),
 'std_fit_time': array([ 4.88050714, 11.09951783, 13.19053357, 17.77356175, 19.45715094,
        17.99466255, 12.23608985]),
 'mean_score_time': array([0.05966644, 0.12088308, 0.15752845, 0.17659845, 0.18135972,
        0.1655046 , 0.1296598 ]),
 'std_score_time': array([0.00988526, 0.02760326, 0.03583377, 0.0530052 , 0.06036735,
        0.044415  , 0.02020915]),
 'param_max_depth': masked_array(data=[3, 6, 8, 10, 12, 16, 20],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 3},
  {'max_depth': 6},
  {'max_depth': 8},
  {'max_depth': 10},
  {'max_depth': 12},
  {'max_depth': 16},
  {'max_depth': 20}],
 'split0_test_score': array([0.76548747, 0.7560088 , 0.76819567, 0.77361205, 0.77335816,
        0.77335816, 0.77335816]),
 'split1_test_score': array([0.87593094, 0.

In [12]:
param_grid

{'max_depth': [3, 6, 8, 10, 12, 16]}

In [13]:
# summarize results
print("Best: %f using %s" % ( grid_result.best_score_, grid_result.best_params_))



Best: 0.842756 using {'max_depth': 8}


In [14]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
                                 
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
                                
                

0.836391 (0.047333) with: {'max_depth': 3}
0.841452 (0.053723) with: {'max_depth': 6}
0.842756 (0.051317) with: {'max_depth': 8}
0.835697 (0.047306) with: {'max_depth': 10}
0.833548 (0.048347) with: {'max_depth': 12}
0.840234 (0.049060) with: {'max_depth': 16}


In [31]:
pyplot.errorbar(paramToTuneList, means, yerr=stds)


pyplot.title(f"XGBoost {tuneName} vs {scoreX} for {strategyName}")
pyplot.xlabel(tuneName)
pyplot.ylabel(scoreX)
pyplot.savefig(f'{tuneName}{st}To{ed}_{scoreX}-{strategyName}.png')